# Load Libraries into PostGIS

Loads cleaned and enriched Berlin libraries into Postgres/PostGIS.
Safe to re-run (ON CONFLICT DO NOTHING).


In [3]:
import geopandas as gpd
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

In [8]:
# Load the unified dataset into the Postgres database and runs QA checks.
library_df_district = gpd.read_file("../sources/libraries_db_unified.geojson")

library_df_district.head()

,name,amenity,library_id,operator_type,operator,street,housenumber,postcode,city,country,...,final_email,final_phone,website_url,longitude,latitude,district,neighbourhood,neighbourhood_id,district_id,geometry
0,Bruno-Lösche-Bibliothek,library,29071031,None,None,Perleberger Straße,33,10559,Berlin,DE,...,None,+49 30901833025,http://www.berlin.de/stadtbibliothek-mitte/bib...,13.347514,52.531245,Mitte,Moabit,0102,11001001,POINT (13.34751 52.53124)
1,Anton-Saefkow-Bibliothek,library,60848456,None,None,Anton-Saefkow-Platz,14,10369,Berlin,DE,...,None,+4930902963773,http://www.berlin.de/ba-lichtenberg/auf-einen-...,13.470838,52.530777,Lichtenberg,Fennpfuhl,1111,11011011,POINT (13.47084 52.53078)
2,Stadtteilbibliothek Erich Weinert,library,203557001,None,None,Helene-Weigel-Platz,4,12681,Berlin,DE,...,None,+49 30 5429251,https://www.berlin.de/bibliotheken-mh/biblioth...,13.538715,52.528158,Marzahn-Hellersdorf,Marzahn,1001,11010010,POINT (13.53872 52.52816)
3,Stadtteilbibliothek Halemweg,library,256922190,None,None,Halemweg,18,13627,Berlin,DE,...,None,None,None,13.287186,52.537504,Charlottenburg-Wilmersdorf,Charlottenburg-Nord,0406,11004004,POINT (13.28719 52.5375)
4,Bezirkszentralbibliothek Spandau,library,257708789,None,None,Carl-Schurz-Straße,13,13597,Berlin,DE,...,bibliothek@ba-spandau.berlin.de,+49 30 90279 5537,https://www.berlin.de/stadtbibliothek-spandau/...,13.201386,52.536133,Spandau,Spandau,0501,11005005,POINT (13.20139 52.53613)


In [9]:
library_df_district.columns

Index(['name', 'amenity', 'library_id', 'operator_type', 'operator', 'street',
       'housenumber', 'postcode', 'city', 'country', 'opening_hours',
       'wheelchair_accessible', 'toilets_wheelchair', 'level',
       'internet_access', 'isil_code', 'email', 'phone', 'contact_phone',
       'website', 'contact_website', 'final_email', 'final_phone',
       'website_url', 'longitude', 'latitude', 'district', 'neighbourhood',
       'neighbourhood_id', 'district_id', 'geometry'],
      dtype='object')

### **Populate Database**


DBeaver requires no host connenction info rather only SQL create table and then --> play button for excution

otherwise, in jupter notebook, i need to host connect with my username , password etc as it is below.

In [ ]:

# Database connection parameters

# Credentials
user_name=''
password=''

# Conection
host = ''
port = ''
database = ''
schema=''

# Connection to db after you opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

In [ ]:
# Drop the Old Table (Important!)

from sqlalchemy import text

drop_table_query = f"DROP TABLE IF EXISTS {schema}.libraries CASCADE;"
with engine.connect() as conn:
    conn.execute(text(drop_table_query))
    conn.commit()
print("Old 'libraries' table dropped successfully.")

In [ ]:
#this is where you create table with constraints and references first
# SQL query to create the libraries table with constraints and foreign key

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.libraries (
    -- 🔑 Primary key & IDs
    library_id BIGINT PRIMARY KEY,
    
    -- 📚 Library details
    name VARCHAR(255) NOT NULL,
    amenity VARCHAR(50) NOT NULL,
    operator_type VARCHAR(100),
    operator VARCHAR(500),
    
    -- 🏠 Address fields
    street VARCHAR(150),
    housenumber VARCHAR(10),
    postcode VARCHAR(10) NOT NULL,
    city VARCHAR(50) NOT NULL,
    country VARCHAR(50) NOT NULL,
    
    -- 🕑 Opening & accessibility
    opening_hours VARCHAR(255),
    wheelchair_accessible VARCHAR(10),
    toilets_wheelchair VARCHAR(10),
    level VARCHAR(10),
    internet_access VARCHAR(100),
    
    -- 📑 Standards & contact
    isil_code VARCHAR(100),
    email VARCHAR(255),
    phone VARCHAR(100),
    contact_phone VARCHAR(100),
    website VARCHAR(255),
    contact_website VARCHAR(255),
    final_email VARCHAR(255),
    final_phone VARCHAR(100),
    website_url VARCHAR(255),
    
    -- 📍 Location
    longitude NUMERIC(9,6) NOT NULL,
    latitude NUMERIC(9,6) NOT NULL,
    
    -- 🗺️ Administrative / relational
    district VARCHAR(100) NOT NULL,
    neighbourhood VARCHAR(100),
    neighbourhood_id VARCHAR(100),
    district_id VARCHAR(8) NOT NULL,
    
    -- 🌐 Geometry for PostGIS
    geometry GEOMETRY(Point, 4326),

    -- 🔗 Foreign key constraint
    CONSTRAINT district_id_fk
        FOREIGN KEY (district_id)
        REFERENCES berlin_source_data.districts(district_id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);
"""

# Execute the query to create empty table
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction

In [ ]:
# Insert the Data
#  Send the DataFrame to the database using .to_sql()
library_df_district.to_sql(
    'libraries',      
    engine,
    schema=schema,
    if_exists='append', # ✅ keeps table, just inserts data
    index=False
)


print("DataFrame (with WKT geometryand district IDs) successfully loaded into PostgreSQL!")

In [ ]:
##let's query test data!
query = f"""
SELECT * 
from berlin_source_data.libraries
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df.head(2)